Berdasarkan isu [#158](https://github.com/hidrokit/hidrokit/issues/158): Statistik Dasar (kurtosis, stdev, skew)

Referensi Isu:
- Soetopo, W., Montarcih, L., Press, U. B., & Media, U. (2017). Rekayasa Statistika untuk Teknik Pengairan. Universitas Brawijaya Press. https://books.google.co.id/books?id=TzVTDwAAQBAJ

Deskripsi Isu:
- Mencari nilai parameter statistik dasar berupa kurtosis, skew, standar deviasi, yang dibutuhkan untuk penentuan jenis sebaran yang sesuai dengan data.

Diskusi Isu:
- [#163](https://github.com/hidrokit/hidrokit/discussions/163): Apa rumus yang tepat dalam mencari nilai kurtosis dari data?
- [#157](https://github.com/hidrokit/hidrokit/discussions/157): Referensi Lanjutan untuk Jenis Sebaran?

Strategi:
- Membuat opsi antara menggunakan formula paket external atau penggunaan perhitungan manual.
- Menampilkan jenis sebaran yang sesuai dan syarat-syaratnya. 

# PERSIAPAN DAN DATASET

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# contoh data diambil dari buku
# Rekayasa Statistika untuk Teknik Pengairan h.32-33

_H = np.array(
    [176, 102, 276, 254, 122, 320, 297, 203, 245, 268, 
     236, 210, 214, 151, 277, 149, 245, 154, 261, 255]
)

data = pd.DataFrame(data=_H, columns=['H'])
data

,H
0,176
1,102
2,276
3,254
4,122
5,320
6,297
7,203
8,245
9,268


# KODE

In [ ]:
def _var(x):
    n = x.size
    return ((x-x.mean())**2).sum()/(n-1)

def _std(x):
    return np.sqrt(_var(x))

def _momen(x, r):
    n = x.size
    return 1/n * ((x-x.mean())**r).sum()

def _skew(x):
    n = x.size
    return n**2 / ((n-1)*(n-2)) * _momen(x, 3) / _std(x)**3

def _kurt(x):
    n = x.size
    return n**3 / ((n-1)*(n-2)*(n-3)) * _momen(x, 4) / _std(x)**4

def _Cv(x):
    return _std(x) / x.mean()

def calc_coef(x):
    """Return (Cv, Cs, Ck)"""
    return (_Cv(x), _skew(x), _kurt(x))

def check_distribution(x, show_stat=False, show_detail=False):

    Cv, Cs, Ck = calc_coef(x)

    if show_stat:
        print(
            f'Cv = {Cv:.5f}',
            f'Cs = {Cs:.5f}',
            f'Ck = {Ck:.5f}',
            sep='\n', end='\n'
        )

    b_normal = True if np.isclose(Cs, 0, atol=0.1) and np.isclose(Ck, 3, atol=0.1) else False
    b_lognormal = True if np.isclose(Cs, 3, atol=0.1) and np.greater(Cs, 0) and np.isclose(Cs, 3*Cv, atol=0.1) else False
    b_gumbel = True if np.isclose(Cs, 1.1396, atol=0.0001) and np.isclose(Ck, 5.4002, atol=0.0001) else False
    b_logpearson = True #selalu benar terlepas nilai Cv, Cs, Ck

    _kriteria = lambda x: "Memenuhi" if x else "Tidak Memenuhi"

    print(
        f'{"Normal":<20}: {_kriteria(b_normal)}',
        f'{"Log Normal":<20}: {_kriteria(b_lognormal)}',
        f'{"Gumbel Tipe I":<20}: {_kriteria(b_gumbel)}',
        f'{"Log Pearson Tipe III":<20}: {_kriteria(b_logpearson)}',
        sep='\n', end='\n'
    )

    if show_detail:
        print(
            '-----------------------------------------------',
            '> Distribusi Normal [syarat](nilai)',
            f'[Cs ~ 0](Cs = {Cs:.5f})',
            f'[Ck ~ 3](Ck = {Ck:.5f})',
            '> Log Normal',
            f'[Cs ~ 3](Cs = {Cs:.5f})',
            f'[Cs > 0](Cs = {Cs:.5f})',
            f'[Cs ~ 3Cv](Cs = {Cs:.5f} ~ 3Cv = {3*Cv:.5f})',
            '> Gumbel Tipe I',
            f'[Cs ~ 1.1396](Cs = {Cs:.5f})',
            f'[Ck ~ 5.4002](Ck = {Ck:.5f})',
            '> Log Pearson Tipe III',
            'Tidak memiliki ciri khas (Cs/Ck/Cv = Bebas)',
            '-----------------------------------------------',
            sep='\n', end='\n'
        )

# FUNGSI

## Fungsi Parameter Statistik `_var(), _std(), _momen(), _skew(), _kurt(), _Cv()`

Pada modul ini dibuat fungsi untuk menghitung parameter statistik berupa:
- `_var(x)`: menghitung ragam (_variance_)
- `_std(x)`: menghitung simpangan baku / standar deviasi (_standard deviation_)
- `_momen(x, r)`: menghitung momen dengan besar momen `r`
- `_skew(x)`: menghitung kepencengan (_skewness_)
- `_kurt(x)`: menghitung ukuran kepuncakkan (_kurtosis_)
- `_Cv(x)`: menghitung koefisien _variance_

dengan `x` merupakan `array` dan `r` merupakan besar momen. 

In [ ]:
# karena fungsi parameter statistik tersebut
# juga sudah tersedia di numpy dan scipy
# maka akan dibandingkan hasilnya

# variance
print('var [hk == numpy]:', _var(data.H) == np.var(data.H, ddof=1))

# standard deviation
print('std [hk == numpy]:', _std(data.H) == np.std(data.H, ddof=1))

# skewness 
print('skew [hk == scipy]:', _skew(data.H) == stats.skew(data.H, bias=False))

# kurtosis
print('kurt [hk == scipy]:', _kurt(data.H) == stats.kurtosis(data.H, bias=False))
print(
    f'kurt [hk] = {_kurt(data.H)}',
    f'kurt [scipy] = {stats.kurtosis(data.H, bias=False)}', sep='\n'
)

var [hk == numpy]: True
std [hk == numpy]: True
skew [hk == scipy]: True
kurt [hk == scipy]: False
kurt [hk] = 2.665512982820046
kurt [scipy] = -0.7404270543134617


Disimpulkan bahwa untuk perhitungan kurtosis memiliki perbedaan antara menggunakan formula dari referensi dengan paket `scipy`. 

## Fungsi `calc_coef(x)`:

Fungsi `calc_coef(...)` digunakan untuk menghitung nilai koefisien variance ($C_v$), koefisien skewness ($C_s$), koefisien kurtosis ($C_k$). Argumen yang diminta adalah `x` berupa `array`. Catatan: Perhitungan koefisien skewness dan kurtosis merupakan perhitungan parameter statistik skewness dan kurtosis. 


In [ ]:
Cv, Cs, Ck = calc_coef(data.H)
print(Cv, Cs, Ck)

0.27539643336275177 -0.4406607753164903 2.665512982820046


## Fungsi `check_distribution(x, show_stat=False)`

Fungsi `check_distribution()` digunakan untuk memeriksa jenis sebaran dari data. 

- Argumen Posisi:
    - `x`: `array`
- Argumen Opsional:
    - `show_stat=False`: Jika `True` maka menampilkan nilai Cv, Cs, dan Ck.
    - `show_detail=False`: Jika `True` maka menampilkan persyaratan jenis sebaran (distribusi). 

In [ ]:
check_distribution(data.H)

Normal              : Tidak Memenuhi
Log Normal          : Tidak Memenuhi
Gumbel Tipe I       : Tidak Memenuhi
Log Pearson Tipe III: Memenuhi


In [ ]:
check_distribution(data.H, show_stat=True)

Cv = 0.27540
Cs = -0.44066
Ck = 2.66551
Normal              : Tidak Memenuhi
Log Normal          : Tidak Memenuhi
Gumbel Tipe I       : Tidak Memenuhi
Log Pearson Tipe III: Memenuhi


In [ ]:
check_distribution(data.H, show_detail=True)

Normal              : Tidak Memenuhi
Log Normal          : Tidak Memenuhi
Gumbel Tipe I       : Tidak Memenuhi
Log Pearson Tipe III: Memenuhi
-----------------------------------------------
> Distribusi Normal [syarat](nilai)
[Cs ~ 0](Cs = -0.44066)
[Ck ~ 3](Ck = 2.66551)
> Log Normal
[Cs ~ 3](Cs = -0.44066)
[Cs > 0](Cs = -0.44066)
[Cs ~ 3Cv](Cs = -0.44066 ~ 3Cv = 0.82619)
> Gumbel Tipe I
[Cs ~ 1.1396](Cs = -0.44066)
[Ck ~ 5.4002](Ck = 2.66551)
> Log Pearson Tipe III
Tidak memiliki ciri khas (Cs/Ck/Cv = Bebas)
-----------------------------------------------


# Changelog

```
- 20220324 - 1.1.0 - tambah argumen show_detail=False untuk check_distribution()
- 20220304 - 1.0.0 - Initial
```

#### Copyright &copy; 2022 [Taruma Sakti Megariansyah](https://taruma.github.io)

Source code in this notebook is licensed under a [MIT License](https://choosealicense.com/licenses/mit/). Data in this notebook is licensed under a [Creative Common Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/). 
